<a href="https://colab.research.google.com/github/Swethaa-sri/Task_cl/blob/main/Assignment_4_Boston_Housing_Dataset_D15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Importing numpy library

In [1]:
import numpy as np

#### Importing phi from train data set

In [2]:
phi = np.loadtxt('train.csv', dtype = 'float', delimiter = ',', skiprows = 1, usecols = tuple(range(1,14))  )

In [ ]:
phi

#### Import y from train data set

In [4]:
y = np.loadtxt('train.csv', dtype = 'float', delimiter = ',', skiprows = 1, usecols = 14, ndmin = 2)

In [ ]:
print(y)

#### Import phi_test from test data set

In [6]:
phi_test = np.loadtxt('test.csv', dtype = 'float', delimiter = ',', skiprows = 1, usecols = tuple(range(1,14)))

In [ ]:
print(phi_test)

#### Add a column of 1s to the right of phi and phi_test

In [7]:
phi_test.shape

(105, 13)

In [8]:
phi.shape

(400, 13)

In [9]:
phi_test = np.concatenate((phi_test, np.ones((105,1))),1)

In [10]:
phi_test.shape

(105, 14)

In [11]:
phi = np.concatenate((phi, np.ones((400,1))),1)

In [12]:
phi.shape

(400, 14)

#### Min Max scaling for phi and phi_test 

In [13]:
for i in range(0, 13):
  col_max = max(phi[:,i])
  col_min = min(phi[:,i])
  phi[:,i] = (phi[:,i] - col_min)/(col_max - col_min)
  phi_test[:,i] = (phi_test[:,i] - col_min)/(col_max - col_min)

#### Log scaling on y

In [14]:
y = np.log(y)

#### Function to calculate change in error function

In [15]:
def d_w(p, phi, w):
  if p == 2:
    deltaw = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambd * p * np.power(np.absolute(w), (p - 1)))
  if p < 2 and p > 1:
        deltaw = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambd * p * np.power(np.absolute(w), (p - 1)) * np.sign(w))
  return deltaw

#### Dictionary containing filenames as keys and p as values

In [16]:
filenames = {'output.csv': 2.0,
             'output_p1.csv': 1.75,
             'output_p2.csv': 1.5,
             'output_p3.csv': 1.3
             }


In [17]:
# For each item in this dictionary
for (fname, p) in filenames.items():
    # Set initial w to zeros
    w = np.zeros((14, 1))
    # Hyperparameter lambda value
    lambd = 0.2
    # Maximum step size
    t = 0.00012
    # Calculate new value of w
    w_new = w - t * d_w(p, phi, w)
    i = 0
    # Repeat steps until error between consecutive w is less than threshold
    while(np.linalg.norm(w_new-w) > 10 ** -10):
        w = w_new
        w_new = w - t * d_w(p, phi, w)
        i = i + 1
    # Load values of id
    id_test = np.loadtxt('test.csv', dtype='int', delimiter=',',
                         skiprows=1, usecols=0, ndmin=2)
    # Calculate y for test data
    y_test = np.exp(np.dot(phi_test, w_new))

    # Save the ids and y
    np.savetxt(fname, np.concatenate((id_test, y_test), axis=1),
               delimiter=',', fmt=['%d', '%f'], header='ID,MEDV', comments='')